In [41]:
import pandas as pd

data = pd.read_excel("241220_apify.xlsx",
                     sheet_name="actor_Instagram Profile Scraper"
                     )
data = data.loc[:, ~data.columns.str.contains('latestIgtvVideos')]
data = data.loc[(data.followersCount > 2000) & (data.followersCount < 10000)]

In [46]:
df = pd.DataFrame({'url':data["inputUrl"],
                   'username':data["username"],
                   'biography':data["biography"],
                   'followers':data["followersCount"],
                   'following':data["followsCount"],
                   'businessCategory':data["businessCategoryName"],
                   'externalUrl':data["externalUrl"],
                   }).reset_index(drop=True)
df.head()

,url,username,biography,followers,following,businessCategory,externalUrl
0,https://www.instagram.com/andreapachetti/,andreapachetti,📀 Producer - Sound Engineer - Songwriter,3108.0,889.0,Music Producer,https://spotify.link/5RM8UekXYDb
1,https://www.instagram.com/marco_torosantucci/,marco_torosantucci,📈Trader & Imprenditore\n✨ Founder @lumix_clean...,9559.0,2559.0,Entrepreneur,https://www.lumixcleaner.it/
2,https://www.instagram.com/crazytrip_bolgia/,crazytrip_bolgia,🔥Il tuo PR di fiducia @bolgia.official \n🎧 Tra...,2084.0,3399.0,Event Planner,http://wa.me/+393479650804
3,https://www.instagram.com/headbangerspubmilano/,headbangerspubmilano,Il pub metal a Milano 🤘🍻🤘,7556.0,2210.0,"None,Dance & Night Club",http://www.headbangerspub.com/
4,https://www.instagram.com/edops/,edops,"Autore, host del podcast @hacking_creativity e...",2463.0,1241.0,Video creator,http://bit.ly/hc-podcast


In [92]:
for i in range(0, 12):
    try:
        df[f"post_{i}"] = data[f"latestPosts/{i}/displayUrl"].tolist()
        df[f"caption_{i}"] = data[f"latestPosts/{i}/caption"].tolist()
    except Exception as e:
        print('Unable to find column: ',e)
        continue

Unable to find column:  'latestPosts/2/caption'
Unable to find column:  'latestPosts/6/caption'
Unable to find column:  'latestPosts/8/displayUrl'


In [100]:
from together import Together

TOGETHER_API_KEY = "5a532872525382e32ebc396c6cc682d3b8d8d5ea428ef9468404286bb1417f2c"
client = Together(api_key=TOGETHER_API_KEY)

getDescriptionPrompt = open("../prompts/describe_image.txt", "r").read()
imageUrl = df.post_0[0]


response = client.chat.completions.create(
    model="meta-llama/Llama-Vision-Free",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": getDescriptionPrompt},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": imageUrl,
                    },
                },
            ],
        }
    ],
)
print(response.choices[0].message.content)

This image depicts a young woman sprinting down a long and narrow urban alleyway, with a dramatic and intense atmosphere. The woman, dressed in a white collared shirt, blue skirt, and black leather jacket, is captured in mid-stride, her face blurred as she gazes downward. The alleyway's gray concrete walls and floor, punctuated by a few windows and doors, create a sense of confinement and isolation.

The image's overall mood is one of urgency and tension, as if the woman is fleeing from an unknown threat or danger. The blurred motion and shallow depth of field further emphasize the sense of movement and urgency, drawing the viewer's attention to the woman's frantic pace.


latestPosts/0/caption  
latestPosts/0/displayUrl  
latestPosts/0/url  
latestPosts/0/type  
latestPosts/0/images/4  